In [3]:
!pip install LightGBM
!pip install CatBoost
!pip install xgboost
!pip install numpy
!pip install sklearn
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd

from sklearn.metrics import mean_squared_error
import numpy as np

from catboost import CatBoostRegressor
from xgboost import XGBRFRegressor
from lightgbm import LGBMRegressor

In [ ]:
BASE_PATH = 'good_dataset.csv'
TSFEL_DATA_PATH = None
# Не обращай внимания на некоторые конкретные номера фич в фичах с перемножением.
# В Этом списке я задаю фичи (далее общие фичи) которые пересоздаются для тренировочного и для тестового датасетов.
# В общие фичи я кладу первый участок для обучения и второй участок для тестирования
# Закоментированные фичи были пропущенны авторами в датасете, к сожалению
BASE_FEATURES = [
    'Мощность МПСИ',
    'Мощность МШЦ',
    'Ток МПСИ',
    'Ток МШЦ',
    'Исходное питание МПСИ',
    'Возврат руды МПСИ',
    'Общее питание МПСИ',
    'Расход воды МПСИ PV',
    'Расход воды МПСИ SP',
    'Расход воды МПСИ CV',
    'факт соотношение руда/вода МПСИ',
    'Давление на подшипник МПСИ загрузка',
    'Давление на подшипник МПСИ разгрузка',
    'Поток',
    'Поток_×_Мощность',
    'Поток_×_Ток',
    'Поток_×_Питание МПСИ 2',
    'Поток_×_Расход воды МПСИ 2 PV',
    'Поток_×_Расход воды МПСИ 2 SP',
    'Поток_×_Расход воды МПСИ 2 CV',
    'Поток_×_Давление на подшипник МПСИ 2 загрузка',
    'Поток_×_Давление на подшипник МПСИ 2 разгрузка',
    'sin_поток',
    'поток_квадрат',
    'поток_логарифм',
    'поток_лаг_1',
    'поток_лаг_2',
    'поток_лаг_3',
    'поток_лаг_4',
    'поток_лаг_5',
    'поток_лаг_6',
    'поток_лаг_7',
    'поток_лаг_8',
    'поток_лаг_9',
    'поток_лаг_10',
    'поток_лаг_11',
    'поток_лаг_12',
    'поток_лаг_13',
    'поток_лаг_14',
    'поток_лаг_15',
    'поток_лаг_16',
    'поток_rolling_10_mean',
    'поток_rolling_10_std',
    'поток_rolling_10_min',
    'поток_rolling_10_max',
    'поток_rolling_10_median',
    'поток_rolling_10_sum',
    'поток_rolling_20_mean',
    'поток_rolling_20_std',
    'поток_rolling_20_min',
    'поток_rolling_20_max',
    'поток_rolling_20_median',
    'поток_rolling_20_sum',
    'поток_rolling_30_mean',
    'поток_rolling_30_std',
    'поток_rolling_30_min',
    'поток_rolling_30_max',
    'поток_rolling_30_median',
    'поток_rolling_30_sum',
    'cos_поток',
    'tg_поток',
    'Гранулометрия в сливе',
    'Cu',
    'S',
    'Zn',
    'Переработка_руды_ВМТ',
    'Влажность',
    'Переработка_руды_СМТ',
    #'Температура масла основной маслостанции подача МПСИ',
    #'Температура масла основной маслостанции слив МПСИ',
    #'Температура масла маслостанции электродвигатель МПСИ',
    #'Температура масла редуктора МПСИ',
    #'Температура масла основной маслостанции подача МШЦ',
    #'Температура масла основной маслостанции слив МШЦ',
    #'Температура масла маслостанции электродвигатель МШЦ',
    #'Температура масла редуктора МШЦ',
    'Расход извести МШЦ',
    'Уровень в зумпфе',
    'Обороты насоса',
    'Давление в ГЦ насоса',
    'Плотность слива ГЦ'
]

TARGET = 'Гранулометрия 2 %'
TARGET_1 = 'Гранулометрия 1 %'


In [7]:
train = pd.read_csv(BASE_PATH,delimiter=',')

,Время,Мощность МПСИ 1 кВт,Мощность МПСИ 2 кВт,Мощность МШЦ 1 кВт,Мощность МШЦ 2 кВт,Ток МПСИ 1 А,Ток МПСИ 2 А,Ток МШЦ 1 А,Ток МШЦ 2 А,Исходное питание МПСИ 1 т/ч,...,Zn_1,S_1,Гранулометрия_1,Переработка_руды_ВМТ_2,Влажность_2,Переработка_руды_СМТ_2,Cu_2,Zn_2,S_2,Гранулометрия_2
0,2022-10-14 09:41:00,2775.416504,2867.076172,3362.618408,3211.990723,311.261475,313.438446,367.854431,351.281342,419.317566,...,0.18,26.29,82.8,6349.0,2.6,6183.926,0.84,0.16,22.05,83.5
1,2022-10-14 09:42:00,2782.308105,2863.872559,3360.184326,3213.338867,310.980560,312.871735,367.691681,351.527618,366.358826,...,0.18,26.29,82.8,6349.0,2.6,6183.926,0.84,0.16,22.05,83.5
2,2022-10-14 09:43:00,2790.591797,2853.717529,3369.075195,3214.086670,312.216309,312.942566,368.021637,351.625549,407.251831,...,0.18,26.29,82.8,6349.0,2.6,6183.926,0.84,0.16,22.05,83.5
3,2022-10-14 09:44:00,2800.868652,2873.104492,3374.479736,3226.053223,313.523560,314.291443,368.773621,352.571655,444.706696,...,0.18,26.29,82.8,6349.0,2.6,6183.926,0.84,0.16,22.05,83.5
4,2022-10-14 09:45:00,2796.378174,2892.354492,3374.363281,3229.224365,312.518616,315.674683,369.020020,352.981232,362.717834,...,0.18,26.29,82.8,6349.0,2.6,6183.926,0.84,0.16,22.05,83.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287105,2023-05-01 20:31:00,2467.834229,1788.085571,3444.008545,2993.943115,276.693848,195.216980,376.638794,327.802795,387.366364,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287106,2023-05-01 20:32:00,2484.389160,1781.225708,3454.210693,3004.321289,278.518982,195.231537,376.507843,327.850769,391.852081,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287107,2023-05-01 20:33:00,2497.712646,1779.578979,3439.377686,2992.487793,279.480835,194.519180,375.815826,327.353119,341.617462,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287108,2023-05-01 20:34:00,2497.078369,1762.489746,3435.793701,2985.412598,279.301666,192.897522,374.983582,326.438446,282.291565,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Создаю фичи для обучения
train['Поток_×_Мощность'] = train['Поток 1 л/мин'] * train['Мощность МПСИ 1 кВт']
train['Поток_×_Ток'] = train['Поток 1 л/мин'] * train['Ток МШЦ 1 А']
train['Поток_×_Питание МПСИ 2'] = train['Поток 1 л/мин'] * train['Исходное питание МПСИ 1 т/ч']
train['Поток_×_Расход воды МПСИ 2 PV'] = train['Поток 1 л/мин'] * train['Расход воды МПСИ 1 PV м3/ч']
train['Поток_×_Расход воды МПСИ 2 SP'] = train['Поток 1 л/мин'] * train['Расход воды МПСИ 1 SP м3/ч']
train['Поток_×_Расход воды МПСИ 2 CV'] = train['Поток 1 л/мин'] * train['Расход воды МПСИ 1 CV %']
train['Поток_×_Давление на подшипник МПСИ 2 загрузка'] = train['Поток 1 л/мин'] * train['Давление на подшипник МПСИ 1 загрузка Бар']
train['Поток_×_Давление на подшипник МПСИ 2 разгрузка'] = train['Поток 1 л/мин'] * train['Давление на подшипник МПСИ 1 разгрузка Бар']
train['sin_поток'] = np.sin(train['Поток 1 л/мин'])

train['cos_поток'] = np.cos(train['Поток 1 л/мин'])
train['tg_поток'] = np.sin(train['Поток 1 л/мин'])/np.cos(train['Поток 1 л/мин'])

train['поток_квадрат'] = train['Поток 1 л/мин'] ** 2
train['поток_логарифм'] = np.log(train['Поток 1 л/мин'] + 1e-6)
train['поток_лаг_1'] = train['Поток 1 л/мин'].shift(1)
train['поток_лаг_2'] = train['Поток 1 л/мин'].shift(2)
train['поток_лаг_3'] = train['Поток 1 л/мин'].shift(3)
train['поток_лаг_4'] = train['Поток 1 л/мин'].shift(4)
train['поток_лаг_5'] = train['Поток 1 л/мин'].shift(5)
train['поток_лаг_6'] = train['Поток 1 л/мин'].shift(6)
train['поток_лаг_7'] = train['Поток 1 л/мин'].shift(7)
train['поток_лаг_8'] = train['Поток 1 л/мин'].shift(8)
train['поток_лаг_9'] = train['Поток 1 л/мин'].shift(9)
train['поток_лаг_10'] = train['Поток 1 л/мин'].shift(10)
train['поток_лаг_11'] = train['Поток 1 л/мин'].shift(11)
train['поток_лаг_12'] = train['Поток 1 л/мин'].shift(12)
train['поток_лаг_13'] = train['Поток 1 л/мин'].shift(13)
train['поток_лаг_14'] = train['Поток 1 л/мин'].shift(14)
train['поток_лаг_15'] = train['Поток 1 л/мин'].shift(15)
train['поток_лаг_16'] = train['Поток 1 л/мин'].shift(16)

train['Мощность МПСИ'] =  train['Мощность МПСИ 1 кВт']
train['Общее питание МПСИ'] = train['Общее питание МПСИ 1 т/ч']
train['Мощность МШЦ'] =  train['Мощность МШЦ 1 кВт']
train['Ток МШЦ'] =  train['Ток МШЦ 1 А']
train['Ток МПСИ'] =  train['Ток МПСИ 1 А']
train['Возврат руды МПСИ'] = train['Возврат руды МПСИ 1 т/ч']
train['Исходное питание МПСИ'] = train['Исходное питание МПСИ 1 т/ч']
train['Расход воды МПСИ PV'] = train['Расход воды МПСИ 1 PV м3/ч']
train['Расход воды МПСИ SP'] = train['Расход воды МПСИ 1 SP м3/ч']
train['Расход воды МПСИ CV'] = train['Расход воды МПСИ 1 CV %']
train['факт соотношение руда/вода МПСИ'] =train['факт соотношение руда/вода МПСИ 1']
train['Давление на подшипник МПСИ загрузка'] =  train['Давление на подшипник МПСИ 1 загрузка Бар']
train['Давление на подшипник МПСИ разгрузка'] =train['Давление на подшипник МПСИ 1 разгрузка Бар']
train['Поток'] = train['Поток 1 л/мин']

window = 10

train['поток_rolling_10_mean'] = train['Поток 1 л/мин'].rolling(window=window).mean()
train['поток_rolling_10_std'] = train['Поток 1 л/мин'].rolling(window=window).std()
train['поток_rolling_10_min'] = train['Поток 1 л/мин'].rolling(window=window).min()
train['поток_rolling_10_max'] = train['Поток 1 л/мин'].rolling(window=window).max()
train['поток_rolling_10_median'] = train['Поток 1 л/мин'].rolling(window=window).median()
train['поток_rolling_10_sum'] = train['Поток 1 л/мин'].rolling(window=window).sum()

window = 20

train['поток_rolling_20_mean'] = train['Поток 1 л/мин'].rolling(window=window).mean()
train['поток_rolling_20_std'] = train['Поток 1 л/мин'].rolling(window=window).std()
train['поток_rolling_20_min'] = train['Поток 1 л/мин'].rolling(window=window).min()
train['поток_rolling_20_max'] = train['Поток 1 л/мин'].rolling(window=window).max()
train['поток_rolling_20_median'] = train['Поток 1 л/мин'].rolling(window=window).median()
train['поток_rolling_20_sum'] = train['Поток 1 л/мин'].rolling(window=window).sum()

window = 30

train['поток_rolling_30_mean'] = train['Поток 1 л/мин'].rolling(window=window).mean()
train['поток_rolling_30_std'] = train['Поток 1 л/мин'].rolling(window=window).std()
train['поток_rolling_30_min'] = train['Поток 1 л/мин'].rolling(window=window).min()
train['поток_rolling_30_max'] = train['Поток 1 л/мин'].rolling(window=window).max()
train['поток_rolling_30_median'] = train['Поток 1 л/мин'].rolling(window=window).median()
train['поток_rolling_30_sum'] = train['Поток 1 л/мин'].rolling(window=window).sum()

train['Cu'] = train['Cu_1']
train['Zn'] = train['Zn_1']
train['S'] = train['S_1']

train['Переработка_руды_СМТ'] = train['Переработка_руды_СМТ_1']
train['Переработка_руды_ВМТ'] = train['Переработка_руды_ВМТ_1']
train['Гранулометрия в сливе'] = train['Гранулометрия_1']
train['Влажность'] = train['Влажность_1']

#train['Температура масла основной маслостанции подача МПСИ'] = train['Температура масла основной маслостанции подача МПСИ 1']
#train['Температура масла основной маслостанции слив МПСИ'] = train['Температура масла основной маслостанции слив МПСИ 1']
#train['Температура масла маслостанции электродвигатель МПСИ'] = train['Температура масла маслостанции электродвигатель МПСИ 1']
#train['Температура масла редуктора МПСИ'] = train['Температура масла редуктора МПСИ 1']
#train['Температура масла основной маслостанции подача МШЦ'] = train['Температура масла основной маслостанции подача МШЦ 1']
#train['Температура масла основной маслостанции слив МШЦ'] = train['Температура масла основной маслостанции слив МШЦ 1']
#train['Температура масла маслостанции электродвигатель МШЦ'] = train['Температура масла маслостанции электродвигатель МШЦ 1']
#train['Температура масла редуктора МШЦ'] = train['Температура масла редуктора МШЦ 1']
#train['Расход извести МШЦ'] = train['Расход извести МШЦ 1 л/ч']
#train['Уровень в зумпфе'] = train['Уровень в зумпфе 1 %']
train['Обороты насоса'] = train['Обороты насоса 1 1 %']
train['Давление в ГЦ насоса'] = train['Давление в ГЦ насоса 1 1 Бар']
#train['Плотность слива ГЦ'] = train['Плотность слива ГЦ 1 кг/л']

train.dropna( inplace=True)

X_test_f = train[BASE_FEATURES]
y_test_f = train[TARGET_1]

In [ ]:
#Создаю фичи для тестирования (Второй участок)

train['Поток_×_Мощность'] = train['Поток 2 л/мин'] * train['Мощность МПСИ 2 кВт']
train['Поток_×_Ток'] = train['Поток 2 л/мин'] * train['Ток МШЦ 2 А']
train['Поток_×_Питание МПСИ 2'] = train['Поток 2 л/мин'] * train['Исходное питание МПСИ 2 т/ч']
train['Поток_×_Расход воды МПСИ 2 PV'] = train['Поток 2 л/мин'] * train['Расход воды МПСИ 2 PV м3/ч']
train['Поток_×_Расход воды МПСИ 2 SP'] = train['Поток 2 л/мин'] * train['Расход воды МПСИ 2 SP м3/ч']
train['Поток_×_Расход воды МПСИ 2 CV'] = train['Поток 2 л/мин'] * train['Расход воды МПСИ 2 CV %']
train['Поток_×_Давление на подшипник МПСИ 2 загрузка'] = train['Поток 2 л/мин'] * train['Давление на подшипник МПСИ 2 загрузка Бар']
train['Поток_×_Давление на подшипник МПСИ 2 разгрузка'] = train['Поток 2 л/мин'] * train['Давление на подшипник МПСИ 2 разгрузка Бар']
train['sin_поток'] = np.sin(train['Поток 2 л/мин'])

train['cos_поток'] = np.cos(train['Поток 2 л/мин'])
train['tg_поток'] = np.sin(train['Поток 2 л/мин'])/np.cos(train['Поток 2 л/мин'])

train['поток_квадрат'] = train['Поток 2 л/мин'] ** 2
train['поток_логарифм'] = np.log(train['Поток 2 л/мин'] + 1e-6)
train['поток_лаг_1'] = train['Поток 2 л/мин'].shift(1)
train['поток_лаг_2'] = train['Поток 2 л/мин'].shift(2)
train['поток_лаг_3'] = train['Поток 2 л/мин'].shift(3)
train['поток_лаг_4'] = train['Поток 2 л/мин'].shift(4)
train['поток_лаг_5'] = train['Поток 2 л/мин'].shift(5)
train['поток_лаг_6'] = train['Поток 2 л/мин'].shift(6)
train['поток_лаг_7'] = train['Поток 2 л/мин'].shift(7)
train['поток_лаг_8'] = train['Поток 2 л/мин'].shift(8)
train['поток_лаг_9'] = train['Поток 2 л/мин'].shift(9)
train['поток_лаг_10'] = train['Поток 2 л/мин'].shift(10)
train['поток_лаг_11'] = train['Поток 2 л/мин'].shift(11)
train['поток_лаг_12'] = train['Поток 2 л/мин'].shift(12)
train['поток_лаг_13'] = train['Поток 2 л/мин'].shift(13)
train['поток_лаг_14'] = train['Поток 2 л/мин'].shift(14)
train['поток_лаг_15'] = train['Поток 2 л/мин'].shift(15)
train['поток_лаг_16'] = train['Поток 2 л/мин'].shift(16)

train['Мощность МПСИ'] =  train['Мощность МПСИ 2 кВт']
train['Общее питание МПСИ'] = train['Общее питание МПСИ 2 т/ч']
train['Мощность МШЦ'] =  train['Мощность МШЦ 2 кВт']
train['Ток МШЦ'] =  train['Ток МШЦ 2 А']
train['Ток МПСИ'] =  train['Ток МПСИ 2 А']
train['Возврат руды МПСИ'] = train['Возврат руды МПСИ 2 т/ч']
train['Исходное питание МПСИ'] = train['Исходное питание МПСИ 2 т/ч']
train['Расход воды МПСИ PV'] = train['Расход воды МПСИ 2 PV м3/ч']
train['Расход воды МПСИ SP'] = train['Расход воды МПСИ 2 SP м3/ч']
train['Расход воды МПСИ CV'] = train['Расход воды МПСИ 2 CV %']
train['факт соотношение руда/вода МПСИ'] =train['факт соотношение руда/вода МПСИ 2']
train['Давление на подшипник МПСИ загрузка'] =  train['Давление на подшипник МПСИ 2 загрузка Бар']
train['Давление на подшипник МПСИ разгрузка'] =train['Давление на подшипник МПСИ 2 разгрузка Бар']
train['Поток'] = train['Поток 2 л/мин']

window = 10

train['поток_rolling_10_mean'] = train['Поток 2 л/мин'].rolling(window=window).mean()
train['поток_rolling_10_std'] = train['Поток 2 л/мин'].rolling(window=window).std()
train['поток_rolling_10_min'] = train['Поток 2 л/мин'].rolling(window=window).min()
train['поток_rolling_10_max'] = train['Поток 2 л/мин'].rolling(window=window).max()
train['поток_rolling_10_median'] = train['Поток 2 л/мин'].rolling(window=window).median()
train['поток_rolling_10_sum'] = train['Поток 2 л/мин'].rolling(window=window).sum()

window = 20

train['поток_rolling_20_mean'] = train['Поток 2 л/мин'].rolling(window=window).mean()
train['поток_rolling_20_std'] = train['Поток 2 л/мин'].rolling(window=window).std()
train['поток_rolling_20_min'] = train['Поток 2 л/мин'].rolling(window=window).min()
train['поток_rolling_20_max'] = train['Поток 2 л/мин'].rolling(window=window).max()
train['поток_rolling_20_median'] = train['Поток 2 л/мин'].rolling(window=window).median()
train['поток_rolling_20_sum'] = train['Поток 2 л/мин'].rolling(window=window).sum()

window = 30

train['поток_rolling_30_mean'] = train['Поток 2 л/мин'].rolling(window=window).mean()
train['поток_rolling_30_std'] = train['Поток 2 л/мин'].rolling(window=window).std()
train['поток_rolling_30_min'] = train['Поток 2 л/мин'].rolling(window=window).min()
train['поток_rolling_30_max'] = train['Поток 2 л/мин'].rolling(window=window).max()
train['поток_rolling_30_median'] = train['Поток 2 л/мин'].rolling(window=window).median()
train['поток_rolling_30_sum'] = train['Поток 2 л/мин'].rolling(window=window).sum()

train['Cu'] = train['Cu_2']
train['Zn'] = train['Zn_2']
train['S'] = train['S_2']

train['Переработка_руды_СМТ'] = train['Переработка_руды_СМТ_2']
train['Переработка_руды_ВМТ'] = train['Переработка_руды_ВМТ_2']
train['Гранулометрия в сливе'] = train['Гранулометрия_2']
train['Влажность'] = train['Влажность_2']

#train['Температура масла основной маслостанции подача МПСИ'] = train['Температура масла основной маслостанции подача МПСИ 2']
#train['Температура масла основной маслостанции слив МПСИ'] = train['Температура масла основной маслостанции слив МПСИ 2']
#train['Температура масла маслостанции электродвигатель МПСИ'] = train['Температура масла маслостанции электродвигатель МПСИ 2']
#train['Температура масла редуктора МПСИ'] = train['Температура масла редуктора МПСИ 1']
#train['Температура масла основной маслостанции подача МШЦ'] = train['Температура масла основной маслостанции подача МШЦ 2']
#train['Температура масла основной маслостанции слив МШЦ'] = train['Температура масла основной маслостанции слив МШЦ 2']
#train['Температура масла маслостанции электродвигатель МШЦ'] = train['Температура масла маслостанции электродвигатель МШЦ 2']
#train['Температура масла редуктора МШЦ'] = train['Температура масла редуктора МШЦ 2']
#train['Расход извести МШЦ'] = train['Расход извести МШЦ 2 л/ч']
#train['Уровень в зумпфе'] = train['Уровень в зумпфе 2 %']
train['Обороты насоса'] = train['Обороты насоса 2 1 %']
train['Давление в ГЦ насоса'] = train['Давление в ГЦ насоса 2 1 Бар']
#train['Плотность слива ГЦ'] = train['Плотность слива ГЦ 2 кг/л']

train.dropna( inplace=True)
X_train = train[BASE_FEATURES]
y_train = train[TARGET]

Количество NaN в y_train: Мощность МПСИ            0
Мощность МШЦ             0
Ток МПСИ                 0
Ток МШЦ                  0
Исходное питание МПСИ    0
                        ..
Расход извести МШЦ       0
Уровень в зумпфе         0
Обороты насоса           0
Давление в ГЦ насоса     0
Плотность слива ГЦ       0
Length: 73, dtype: int64


,Мощность МПСИ,Мощность МШЦ,Ток МПСИ,Ток МШЦ,Исходное питание МПСИ,Возврат руды МПСИ,Общее питание МПСИ,Расход воды МПСИ PV,Расход воды МПСИ SP,Расход воды МПСИ CV,...,S,Zn,Переработка_руды_ВМТ,Влажность,Переработка_руды_СМТ,Расход извести МШЦ,Уровень в зумпфе,Обороты насоса,Давление в ГЦ насоса,Плотность слива ГЦ
406,2748.069092,3220.947266,300.869812,352.150635,654.049561,62.707977,718.688904,137.483734,152.965179,98.148117,...,22.05,0.16,6349.0,2.6,6183.926,475.807526,65.553324,82.560036,0.454781,1.320282
407,2766.530762,3219.926270,302.485718,352.123718,696.840942,60.526993,669.794373,137.693741,162.886505,99.763268,...,22.05,0.16,6349.0,2.6,6183.926,476.531372,67.035970,82.364126,0.432134,1.332508
408,2775.645996,3215.526855,303.364319,352.042206,754.687744,71.241005,718.362793,137.493271,152.902405,99.653641,...,22.05,0.16,6349.0,2.6,6183.926,477.468842,68.794100,83.221436,0.448278,1.333517
409,2791.794678,3219.259277,305.153290,351.746307,695.291809,78.466461,697.997559,137.321945,163.892883,99.892281,...,22.05,0.16,6349.0,2.6,6183.926,479.421143,69.617909,84.264667,0.510246,1.333591
410,2817.431641,3212.525146,308.436035,351.534271,572.538757,87.016273,742.298584,137.381638,167.891541,99.868317,...,22.05,0.16,6349.0,2.6,6183.926,481.161896,69.593592,85.128080,0.565599,1.331181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245565,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,...,23.91,0.18,4233.0,3.1,4101.777,999.434753,67.722535,0.000000,0.000000,1.312666
245566,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,...,23.91,0.18,4233.0,3.1,4101.777,1002.804382,67.916695,0.000000,0.000000,1.312145
245567,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,...,23.91,0.18,4233.0,3.1,4101.777,1004.847473,67.808260,0.000000,0.000000,1.307669
245568,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,...,23.91,0.18,4233.0,3.1,4101.777,1007.559265,67.688033,0.000000,0.000000,1.307871


In [ ]:
import optuna
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor

#Подбираем гипперпараметры для Катбуста
def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0),
        'random_strength': trial.suggest_float('random_strength', 1e-3, 10),
        'verbose': False
    }

    model = CatBoostRegressor(**params)
    model.fit(X_train, y_train)
    return mean_squared_error(y_test_f, model.predict(X_test_f))

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)
params_c = study.best_params

[I 2025-04-13 22:50:38,661] A new study created in memory with name: no-name-313714b8-6a70-4d37-9683-6cf37faf2f31
[I 2025-04-13 22:51:46,786] Trial 0 finished with value: 31.99272804365251 and parameters: {'learning_rate': 0.015459332340051264, 'iterations': 405, 'depth': 9, 'l2_leaf_reg': 2.966886960106758, 'subsample': 0.6019851904474652, 'colsample_bylevel': 0.9282584639965723, 'random_strength': 8.258759538238262}. Best is trial 0 with value: 31.99272804365251.
[I 2025-04-13 22:52:45,786] Trial 1 finished with value: 69.62736191918066 and parameters: {'learning_rate': 0.1576118660345704, 'iterations': 997, 'depth': 5, 'l2_leaf_reg': 3.28962416926852, 'subsample': 0.9225018734434319, 'colsample_bylevel': 0.7279284933843049, 'random_strength': 3.05250867151372}. Best is trial 0 with value: 31.99272804365251.
[I 2025-04-13 22:53:26,187] Trial 2 finished with value: 53.684152932543476 and parameters: {'learning_rate': 0.0188235992222741, 'iterations': 836, 'depth': 5, 'l2_leaf_reg': 4.

In [43]:
import optuna
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 5),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-3, 10),
        'objective': 'reg:squarederror',
        'verbose':False
    }

    model = XGBRegressor(**params)
    model.fit(X_train, y_train)
    return mean_squared_error(y_test_f, model.predict(X_test_f))

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

params_x = study.best_params

[I 2025-04-13 22:54:57,653] A new study created in memory with name: no-name-028f7e57-523e-4771-9309-28e6403083d9
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [22:54:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2025-04-13 22:55:15,672] Trial 0 finished with value: 36.2935030827536 and parameters: {'learning_rate': 0.017738410952509782, 'n_estimators': 163, 'max_depth': 9, 'subsample': 0.8681355856052959, 'colsample_bytree': 0.7651148113632179, 'reg_alpha': 2.9957329880247063, 'reg_lambda': 1.0997790773307248, 'min_child_weight': 5.237931442059147}. Best is trial 0 with value: 36.2935030827536.
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [22:55:17] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2025-04-13 22:55:31,356] Trial 1 finished with value: 72.89403983464682 and parameters

In [44]:
import optuna
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor

def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-3, 10),
        'objective': 'regression',
        'verbose':0
    }

    model = LGBMRegressor(**params)
    model.fit(X_train, y_train)
    return mean_squared_error(y_test_f, model.predict(X_test_f))

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

params_l = study.best_params

[I 2025-04-13 22:57:54,532] A new study created in memory with name: no-name-b7c33250-3f15-4172-922c-a2bd3572c0bf


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


[I 2025-04-13 22:58:03,841] Trial 0 finished with value: 64.9109149748638 and parameters: {'learning_rate': 0.06795095854994206, 'n_estimators': 116, 'subsample': 0.7720290772555934, 'colsample_bytree': 0.7645535756736983, 'reg_alpha': 1.7907212385256783, 'min_child_weight': 6.906059375343776}. Best is trial 0 with value: 64.9109149748638.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


[I 2025-04-13 22:58:18,221] Trial 1 finished with value: 64.23660919403197 and parameters: {'learning_rate': 0.17778072597849356, 'n_estimators': 316, 'subsample': 0.9610946620804395, 'colsample_bytree': 0.6830800852427036, 'reg_alpha': 1.1895868233640694, 'min_child_weight': 7.381528617642122}. Best is trial 1 with value: 64.23660919403197.


...

[I 2025-04-13 22:58:27,605] Trial 2 finished with value: 58.50113628275951 and parameters: {'learning_rate': 0.06647280040153386, 'n_estimators': 157, 'subsample': 0.8861429599356478, 'colsample_bytree': 0.6580995015120991, 'reg_alpha': 1.9667006098242812, 'min_child_weight': 4.624762969112064}. Best is trial 2 with value: 58.50113628275951.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


[I 2025-04-13 22:58:47,850] Trial 3 finished with value: 61.851149259777245 and parameters: {'learning_rate': 0.03037807789500955, 'n_estimators': 399, 'subsample': 0.9496568913831867, 'colsample_bytree': 0.881149227441343, 'reg_alpha': 0.5686981793544515, 'min_child_weight': 1.282134300268223}. Best is trial 2 with value: 58.50113628275951.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


[I 2025-04-13 22:59:06,538] Trial 4 finished with value: 74.13289399431025 and parameters: {'learning_rate': 0.11014379603859582, 'n_estimators': 439, 'subsample': 0.652390101760994, 'colsample_bytree': 0.9973988291436701, 'reg_alpha': 1.3002679520434435, 'min_child_weight': 3.092517081410927}. Best is trial 2 with value: 58.50113628275951.


In [ ]:
#Обучение ансамбля, три катбуста в бэггинге.

from sklearn.metrics import mean_squared_error
from sklearn.utils import resample

X_train_cb1,y_train_cb1 = resample(X_train,y_train,replace=True)
X_train_xgb1,y_train_xgb1 = resample(X_train,y_train,replace=True)
X_train_li1,y_train_li1 = resample(X_train,y_train,replace=True)
X_train_cb2,y_train_cb2 = resample(X_train,y_train,replace=True)
X_train_xgb2,y_train_xgb2 = resample(X_train,y_train,replace=True)
X_train_li2,y_train_li2 = resample(X_train,y_train,replace=True)

all_preds = []

light = LGBMRegressor(**params_l)

xgb = XGBRegressor(**params_x)
cat = CatBoostRegressor(**params_c)

for test in range(10):
    light.fit(X_train_cb1, y_train_cb1)

    cat.fit(X_train_cb2, y_train_cb2)
    xgb.fit(X_train_xgb1, y_train_xgb1)
    #light1.fit(X_train_li1,y_train_li1)
    pred_light1 = light.predict(X_test_f)
    pred_light2 = cat.predict(X_test_f)

    #light2.fit(X_train_li2,y_train_li2)
    pred_light3 = xgb.predict(X_test_f)

    y_pred= np.mean([pred_light3,pred_light2,pred_light1], axis=0)

    all_preds.append(mean_squared_error(y_test_f, y_pred))
    print(f'{test} :  {mean_squared_error(y_test_f, y_pred)}')


print(np.mean(all_preds))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
0:	learn: 36.6386550	total: 337ms	remaining: 2m 16s
1:	learn: 36.0891674	total: 631ms	remaining: 2m 7s
2:	learn: 35.5492355	total: 953ms	remaining: 2m 7s
3:	learn: 35.0096985	total: 1.23s	remaining: 2m 3s
4:	learn: 34.4844170	total: 1.59s	remaining: 2m 7s
5:	learn: 33.9616656	total: 1.87s	remaining: 2m 4s
6:	learn: 33.4525808	total: 2.15s	remaining: 2m 2s
7:	learn: 32.9472191	total: 2.4s	remaining: 1m 58s
8:	learn: 32.4495139	total: 2.65s	remaining: 1m 56s
9:	learn: 31.9657199	total: 2.8s	remaining: 1m 50s
10:	learn: 31.4825599	total: 2.96s	remaining: 1m 46s
11:	learn: 31.0084814	total: 3.11s	remaining: 1m 41s
12:	learn: 30.5520425	total: 3.27s	remaining: 1m 38s
13:	learn: 30.0992411	total: 3.41s	remaining: 1m 35s
14:	learn: 29.6474965	total: 3.63s	remaining: 1m 34s
15:	learn: 29.2048064	total: 3.79s	remaining: 1m 32s
16:	learn: 28.7667836	total: 3.97s	remaining: 1m 30s
17:	learn: 28.3336895	total: 4.13s	re